In [38]:
import numpy as np
import pandas as pd
import os

In [39]:
file_path_label = '../../diabetic/Diabetic-Retinopathy-Detection/archive/trainLabels_cropped.csv'

In [40]:
home_data_label = pd.read_csv(file_path_label)
from learntools.core import binder
binder.bind(globals())
from learntools.machine_learning.ex3 import *

print("Setup Complete")

Setup Complete


In [41]:
import csv
import cv2 as cv
#create a .csv file which contains the features used for the descision tree
# calculate features:
header = ['nameImg', 'meanR', 'meanG', 'meanB', 'devR', 'devG', 'devB', 'class']
# folder with image data
dir_name = '../../diabetic/Diabetic-Retinopathy-Detection/archive/resized_train_cropped/resized_train_cropped/'
directory = os.fsencode(dir_name)
train_Labels = pd.read_csv('../../diabetic/Diabetic-Retinopathy-Detection/archive/trainLabels_cropped.csv')
labels = pd.read_csv('../../diabetic/Diabetic-Retinopathy-Detection/archive/trainLabels_cropped.csv', names=['level'])

In [ ]:
with open('./csv_features', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        filename_base = os.path.basename(filename)
        if filename_base.endswith('.jpeg'):
            index = train_Labels.index
            condition = train_Labels["image"] == filename_base[:-5]
            indx = index[condition]
            classImg = train_Labels["level"].loc[indx]
            level = classImg.tolist()
            img = cv.imread(dir_name+filename_base)
            norm_img = cv.imread(dir_name+filename)
            final_img = cv.normalize(img, norm_img, 0, 255, cv.NORM_MINMAX)
            means, dev = cv.meanStdDev(final_img)
            f1_0 = means[0]
            f1_1 = means[1]
            f1_2 = means[2]
            f2_0 = dev[0]
            f2_1 = dev[1]
            f2_2 = dev[2]
            data = [filename, *f1_0, *f1_1, *f1_2, *f2_0, *f2_1, *f2_2, *level]
            writer.writerow(data)
            continue
        else:
            continue

In [ ]:
csv_data = pd.read_csv('csv_features' )
csv_data.head()

In [ ]:
features = ['meanR', 'meanG', 'meanB', 'devR', 'devG', 'devB']
X = csv_data[features]

y = csv_data['class']